In [1]:

# Get current file path
import inspect
import os
import sys

import numpy as np
from Basilisk.utilities import orbitalMotion, macros

filename = inspect.getframeinfo(inspect.currentframe()).filename
path = os.getcwd()

# Import master classes: simulation base class and scenario base class
sys.path.append(path + '/../')
sys.path.append(path + '/../models')
sys.path.append(path + '/../plotting')
from sim_masters import Sim, Scenario
import cubesat_dynamics, cubesat_fsw
import cubesat_plotting as plots



In [ ]:

# Create your own scenario child class
class scenario_mtb_momentum_management(Sim, Scenario):
    def __init__(self):
        super(scenario_mtb_momentum_management, self).__init__()
        self.name = 'scenario_MtbMomentumManagement'

        # declare additional class variables
        self.msgRecList = {}
        self.sNavTransName = "sNavTransMsg"
        self.attGuidName = "attGuidMsg"

        self.set_DynModel(cubesat_dynamics)
        self.set_FswModel(cubesat_fsw)

        self.configure_initial_conditions()
        self.log_outputs()

    def configure_initial_conditions(self):
        # Configure Dynamics initial conditions
        oe = orbitalMotion.ClassicElements()
        oe.a = 10000000.0  # meters
        oe.e = 0.01
        oe.i = 33.3 * macros.D2R
        oe.Omega = 48.2 * macros.D2R
        oe.omega = 347.8 * macros.D2R
        oe.f = 85.3 * macros.D2R

        DynModels = self.get_DynModel()
        mu = DynModels.gravFactory.gravBodies['earth'].mu
        rN, vN = orbitalMotion.elem2rv(mu, oe)
        orbitalMotion.rv2elem(mu, rN, vN)
        DynModels.scObject.hub.r_CN_NInit = rN  # m   - r_CN_N
        DynModels.scObject.hub.v_CN_NInit = vN  # m/s - v_CN_N
        DynModels.scObject.hub.sigma_BNInit = [[0.1], [0.2], [-0.3]]  # sigma_CN_B
        DynModels.scObject.hub.omega_BN_BInit = [[0.001], [-0.01], [0.03]]  # rad/s - omega_CN_B
    

    def log_outputs(self):
        FswModel = self.get_FswModel()
        DynModel = self.get_DynModel()
        samplingTime = FswModel.processTasksTimeStep
        
        self.rwMotorLog = FswModel.rwMotorTorque.rwMotorTorqueOutMsg.recorder(samplingTime)
        self.attErrorLog = FswModel.trackingError.attGuidOutMsg.recorder(samplingTime)
        self.magLog = DynModel.mag_field_module.envOutMsgs[0].recorder(samplingTime)
        self.tamLog = DynModel.tam_module.tamDataOutMsg.recorder(samplingTime)
        self.tamCommLog = FswModel.tam_com_module.tamOutMsg.recorder(samplingTime)
        self.mtbDipoleCmdsLog = FswModel.mtb_momentum_management_module.mtbCmdOutMsg.recorder(samplingTime)

        self.AddModelToTask(DynModel.taskName, self.rwMotorLog)
        self.AddModelToTask(DynModel.taskName, self.attErrorLog)
        self.AddModelToTask(DynModel.taskName, self.magLog)
        self.AddModelToTask(DynModel.taskName, self.tamLog)
        self.AddModelToTask(DynModel.taskName, self.tamCommLog)
        self.AddModelToTask(DynModel.taskName, self.mtbDipoleCmdsLog)

        # To log the RW information, the following code is used:
        self.mrpLog = DynModel.rwStateEffector.rwSpeedOutMsg.recorder(samplingTime)
        self.AddModelToTask(DynModel.taskName, self.mrpLog)
        self.rwLogs = []
        for item in range(4):
            self.rwLogs.append(DynModel.rwStateEffector.rwOutMsgs[item].recorder(samplingTime))
            self.AddModelToTask(DynModel.taskName, self.rwLogs[item])

    def pull_outputs(self, showPlots):
        dataUsReq = rself.wMotorLog.motorTorque
        dataSigmaBR = self.attErrorLog.sigma_BR
        dataOmegaBR = self.attErrorLog.omega_BR_B
        dataOmegaRW = self.mrpLog.wheelSpeeds
        dataRW = []
        for i in range(4):
            dataRW.append(self.rwLogs[i].u_current)
        dataMagField = self.magLog.magField_N
        dataTam = self.tamLog.tam_S
        dataTamComm = self.tamCommLog.tam_B
        dataMtbDipoleCmds = self.mtbDipoleCmdsLog.mtbDipoleCmds

        # Plot results
        plots.clear_all_plots()
        #   plot the results
        timeData = self.rwMotorLog.times() * macros.NANO2MIN
        plt.close("all")  # clears out plots from earlier test runs

        plots.plot_attitude_error(timeData, dataSigmaBR)
        figureList = {}
        pltName = fileName + "1"
        figureList[pltName] = plt.figure(1)

        plots.plot_rw_motor_torque(timeData, dataUsReq, dataRW, numRW)
        pltName = fileName + "2"
        figureList[pltName] = plt.figure(2)

        plots.plot_rate_error(timeData, dataOmegaBR)
        plots.plot_rw_speeds(timeData, dataOmegaRW, numRW)
        pltName = fileName + "3"
        figureList[pltName] = plt.figure(4)

        plots.plot_magnetic_field(timeData, dataMagField)
        pltName = fileName + "4"
        figureList[pltName] = plt.figure(5)

        plots.plot_data_tam(timeData, dataTam)
        pltName = fileName + "5"
        figureList[pltName] = plt.figure(6)

        plots.plot_data_tam_comm(timeData, dataTamComm)
        pltName = fileName + "6"
        figureList[pltName] = plt.figure(7)

        plots.plot_data_mtb_momentum_management(timeData, dataMtbDipoleCmds, mtbConfigParams.numMTB)
        pltName = fileName + "7"
        figureList[pltName] = plt.figure(8)
        
        if show_plots:
            plt.show()
        else:
            fileName = os.path.basename(os.path.splitext(__file__)[0])
            figureNames = ["attitudeErrorNorm", "rateError"]
            figureList = plots.save_all_plots(fileName, figureNames)

        return figureList


def runScenario(scenario):
    """method to initialize and execute the scenario"""
    simulationTime = macros.min2nano(30.)

    scenario.InitializeSimulation()

    # Configure run time and execute simulation
    scenario.modeRequest = 'mtb_momentum_management'
    scenario.ConfigureStopTime(simulationTime)
    scenario.ExecuteSimulation()

    return


def run(showPlots):
    scenario = scenario_mtb_momentum_management()
    runScenario(scenario)
    figureList = scenario.pull_outputs(showPlots)
    return figureList

if __name__ == "__main__":
    run(True)